<a href="https://colab.research.google.com/github/KorStats/topicmodeling/blob/main/2_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ÌïÑÏöî ÎùºÏù¥Î∏åÎü¨Î¶¨ ÏÑ§Ïπò

In [1]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 480.6/480.6 kB 11.9 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 383.4/383.4 kB 25.0 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 233.6/233.6 kB 9.7 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ

In [2]:
!pip install transformers

In [3]:

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [4]:
# GPU ÏÇ¨Ïö© Ïó¨Î∂Ä ÌôïÏù∏
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# ÏÇ¨Ïö©Ìï† Îç∞Ïù¥ÌÑ∞

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   ·Ñá·Ö•·Ü∏·ÑÖ·Ö≤·ÜØ_·Ñå·Ö°·Üº·ÑÜ·ÖÆ·Ü´.csv   ·Ñâ·Öµ·Ü´·ÑÜ·ÖÆ·Ü´·ÑÄ·Öµ·Ñâ·Ö°_·Ñå·Ö°·Üº·ÑÜ·ÖÆ·Ü´.csv


In [6]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/·Ñâ·Öµ·Ü´·ÑÜ·ÖÆ·Ü´·ÑÄ·Öµ·Ñâ·Ö°_·Ñå·Ö°·Üº·ÑÜ·ÖÆ·Ü´.csv', encoding='utf-8-sig')

In [7]:
df['category'].value_counts()

category
Ï¢ÖÌï©           4000
Í≤ΩÏ†ú           2000
ÏÇ¨Ìöå           2000
Ï†ïÏπò           2000
Ïä§Ìè¨Ï∏†          1000
IT,Í≥ºÌïô         500
ÍµêÏú°/ÏûÖÏãú/NIE     500
Name: count, dtype: int64

In [8]:
data_list = []
for q, label in zip(df['text'], df['category'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(dataset_train)
test=pd.DataFrame(dataset_test)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [9]:
train.shape

(9000, 2)

In [10]:
test.shape

(3000, 2)

# Îç∞Ïù¥ÌÑ∞ Ï†ïÏ†ú

In [11]:
def clean_korean_documents(documents):
    # ÌÖçÏä§Ìä∏ Ï†ïÏ†ú (ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\s„Ñ±-„Ö£Í∞Ä-Ìû£]', '', document)  # ÌäπÏàòÍ∏∞Ìò∏Îßå Ï†úÍ±∞, Ï†ïÍ∑ú ÌëúÌòÑÏãù
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# ÌïúÍ∏ÄÏù¥ ÏïÑÎãå Î¨∏ÏûêÎ•º Ï†úÍ±∞ÌïòÎäî Ìï®Ïàò
def remove_non_korean(text):
    return re.sub(r'[^„Ñ±-„Ö£Í∞Ä-Ìû£\s]', '', text)

In [12]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 3000/3000 [00:00<00:00, 6327.35it/s]


In [13]:
# 3. ÌïôÏäµ Îç∞Ïù¥ÌÑ∞ÏôÄ ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÌïÑÏöîÌïú Ïó¥ Ï∂îÏ∂ú Î∞è ÎùºÎ≤® Ïù∏ÏΩîÎî©
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. ÎùºÎ≤® Ïù∏ÏΩîÎî©
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# KoBERT

In [14]:
# 5. KoBERT Î™®Îç∏ÏùÑ ÏúÑÌïú ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Î∞è Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ±
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [15]:
# ÌÖçÏä§Ìä∏Î•º ÌÜ†ÌÅ∞ÌôîÌïòÎäî Ìï®Ïàò Ï†ïÏùò
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [16]:
# Îç∞Ïù¥ÌÑ∞ÏÖã ÏÉùÏÑ± Î∞è ÌÜ†ÌÅ¨ÎÇòÏù¥Ï¶à
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [17]:
# PyTorch ÌÖêÏÑú ÌòïÏãùÏúºÎ°ú Î≥ÄÌôò
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT Î™®Îç∏ ÏÑ§Ï†ï Î∞è ÌïôÏäµ ÌååÎùºÎØ∏ÌÑ∞ ÏµúÏ†ÅÌôî
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# ÏµúÏ†ÅÏùò ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ Ï∞æÍ∏∞

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Objective Ìï®Ïàò Ï†ïÏùò
def objective(trial):
    # ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÏÉòÌîåÎßÅ
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    # TrainingArguments ÏÑ§Ï†ï
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        per_device_train_batch_size=batch_size,
        num_train_epochs=3,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        report_to="none"
    )

    # Trainer ÏÉùÏÑ±
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1"]


# Optuna ÌÉêÏÉâ
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)

# ÏµúÏ†ÅÏùò ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ Ï∂úÎ†•
print("Best Hyperparameters:", study.best_params)

[I 2025-01-24 05:14:14,959] A new study created in memory with name: no-name-466ec921-253b-4a08-aede-014e3a21449c
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-01-24 05:38:20,690] Trial 0 finished with value: 0.07035175879396985 and parameters: {'learning_rate': 0.0001373254859440294, 'batch_size': 16, 'weight_decay': 0.27937960707000137}. Best is trial 0 with value: 0.07035175879396985.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task t

[I 2025-01-24 06:02:47,755] Trial 1 finished with value: 0.6969678344543013 and parameters: {'learning_rate': 7.146271292714943e-05, 'batch_size': 16, 'weight_decay': 0.0593046598006984}. Best is trial 1 with value: 0.6969678344543013.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to b

# ÏµúÏ†ÅÏùò ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞Î°ú Ïû¨ÌïôÏäµ

In [18]:
#CPU Ï∏°Ï†ï

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï Ìï®Ïàò
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # ÏãúÏûë ÏãúÍ∞Ñ Í∏∞Î°ù
    start_cpu = psutil.cpu_percent(interval=None)  # ÏãúÏûë CPU ÏÇ¨Ïö©Îüâ

    train_function()  # ÌïôÏäµ Ìï®Ïàò Ìò∏Ï∂ú

    end_time = time.time()  # Ï¢ÖÎ£å ÏãúÍ∞Ñ Í∏∞Î°ù
    end_cpu = psutil.cpu_percent(interval=None)  # Ï¢ÖÎ£å CPU ÏÇ¨Ïö©Îüâ

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [19]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` Ìï®Ïàò
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 Ï†êÏàò Í≥ÑÏÇ∞
    return {'eval_f1': f1}  # 'eval_f1' Î∞òÌôò

# OptunaÎ•º ÌôúÏö©Ìïú ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞ ÌäúÎãù
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

'learning_rate': 7.146271292714943e-05, 'batch_size': 16, 'weight_decay': 0.0593046598006984}. Best is trial 1 with value: 0.6969678344543013

In [20]:
# 7. ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú ÌïôÏäµ ÏÑ§Ï†ï
# ÏµúÏ†ÅÏùò ÌïòÏù¥ÌçºÌååÎùºÎØ∏ÌÑ∞Î•º Ï∞æÏùÄ ÌõÑ, training_argsÎ•º ÏÑ§Ï†ïÌï©ÎãàÎã§.
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=3,  # ÏóêÌè¨ÌÅ¨ Ïàò Í≥†Ï†ï
    per_device_train_batch_size=16,  # Î∞∞Ïπò ÌÅ¨Í∏∞ Í≥†Ï†ï
    learning_rate=7.146271292714943e-05,  # ÏµúÏ†ÅÌôîÎêú ÌïôÏäµÎ•†
    weight_decay=0.05930465980069848,  # ÏµúÏ†ÅÌôîÎêú weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
    report_to="none"
)

# ÏµúÏ†ÅÏùò ÌååÎùºÎØ∏ÌÑ∞Î°ú Trainer Íµ¨ÏÑ±
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Ï†ÑÏ≤¥ ÌïôÏäµ Îç∞Ïù¥ÌÑ∞Î°ú ÌèâÍ∞Ä
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# CPU ÏÇ¨Ïö©Îüâ Î∞è ÏÜçÎèÑ Ï∏°Ï†ï
measure_cpu_usage_and_time(trainer7.train)

Elapsed Time: 418.35 seconds
Average CPU Usage: 75.45%


In [22]:
# Î™®Îç∏ Ï†ÄÏû•
joblib.dump(trainer7, '/content/gdrive/MyDrive/trainer_news_text.pkl')

print("Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.")

Î™®Îç∏Ïù¥ Ï†ÄÏû•ÎêòÏóàÏäµÎãàÎã§.


In [ ]:
# 8. Ï†ÄÏû•Îêú Î™®Îç∏Î°ú ÌÖåÏä§Ìä∏ Îç∞Ïù¥ÌÑ∞ ÏòàÏ∏° Î∞è Í≤∞Í≥º Ï†ÄÏû•
# Ï†ÄÏû•Îêú Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞

loaded_model = joblib.load('/content/gdrive/MyDrive/trainer7.pkl')

# Î∂àÎü¨Ïò® Î™®Îç∏ÏùÑ ÏÇ¨Ïö©ÌïòÏó¨ ÏòàÏ∏° ÏàòÌñâ
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [ ]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

text category_prediction
0  Íµ≠ÏÑ∏Ï≤≠ Ï†ÅÎ∞ú ÌÉàÏÑ∏ Îì± ÏïÖÏö© Ïö∞Î†§ ÏÑ∏Ï¢ÖÏ∂©Ï≤≠ÏùºÎ≥¥ Ïû•Ï§ëÏãù Í∏∞Ïûê Ïã†Ïö©Ïπ¥Îìú ÏúÑÏû•Í∞ÄÎßπÏ†ê Ï†ÅÎ∞ú ...                  ÏÇ¨Ìöå
1  ÌïúÏ†Ñ Ï†ïÎ∂ÄÌòÅÏã†Î∞ïÎûåÌöå Ïù¥Ïö©Ìôò Í∏∞Ïûê ÌïúÏ†ÑÏù¥ ÎØ∏ÏÑ∏Î®ºÏßÄÎ•º Ï≤≠ÏÜåÌïòÎäî ÏàòÏÜåÎ≤ÑÏä§Î°ú Ï†ú ÎåÄÌïúÎØºÍµ≠ ...                  Í≤ΩÏ†ú
2  Ïù¥Îç∞ÏùºÎ¶¨ ÌïúÍ¥ëÎ≤î Í∏∞Ïûê Î∏åÎ°úÎìúÎ∞¥ÎìúÎäî ÌÖîÎ†àÏΩ§ ÏôÄ Ìï®Íªò  ÎûúÏºÄÏù¥Î∏îÎßùÏóêÏÑúÎèÑ ÏÇ¨Ïö©ÏûêÏóêÍ≤å ÏÉÅÌïò...               IT,Í≥ºÌïô
3  Ï°∞Î∞∞Ïàô ÏùòÏõê ÏòàÏÇ∞Ï†ïÏ±ÖÌòë Ï¥âÍµ¨ Ï∞©Í≥µ Îí§ Ï∞©Í≥µ ÍπÄÌòïÎØº Í∏∞Ïûê Ï†ÑÎ∂Å Íµ≠ÌöåÏùòÏõêÏ†ÑÎùºÎ∂ÅÎèÑ ÏòàÏÇ∞Ï†ï...                  Ï†ïÏπò
4  ÍµêÏú°Ï≤≠Í≥º ÌïôÍµêÍ∞Ä ÎÇòÏÑú ÌòÑÏã§Ï†Å ÎåÄÏ±Ö ÎßàÎ†®Ìï¥Ïïº ÌôçÏÑ±Ïû• Í∏∞Ïûê Í¥ëÏ£ºÏßÄÏó≠ Ï¥àÎì±ÌïôÍµêÎ≥ëÏÑ§Ïú†ÏπòÏõêÏóê...           ÍµêÏú°/ÏûÖÏãú/NIE
5  Ïπ¥ÎìúÏÇ¨Ïùò ÏàòÏàòÎ£åÏú® Ïù∏ÏÉÅ Î∞©Ïπ®Ïóê ÎåÄÌòï Ïú†ÌÜµÏóÖÍ≥ÑÍ∞Ä ÎßàÏºÄÌåÖ Ï∂ïÏÜåÎ•º Í≤ÄÌÜ†ÌïòÍ≥† ÏûàÎã§ Ïù¥Îç∞ÏùºÎ¶¨...                  Í≤ΩÏ†ú
6  ÌÉúÍ¥ëÍ∑∏Î£π Í≥ÑÏó¥ ÌïúÍµ≠ÏºÄÏù¥Î∏îÌÖîÎ†àÏΩ§ ÎåÄÌëú Ïã†ÎèôÏÑ†Ïù¥ Ïö∞Ï≤¥Íµ≠ÏóêÏÑú ÏïåÎú∞Ìè∞ ÌåêÎß§Î•º ÏãúÏûëÌñàÎã§ Ïùò ...                  Í≤ΩÏ†ú
7  Î∞ïÍ∏∏Ïàò Í∏∞Ïûê Ìè≠Ïóº ÏÜç ÎÜçÏóÖÏù∏ÏóêÍ≤å ÌÅ∞ ÎèÑÏõÄ Ïò§Îäî ÏàòÌôïÏûëÏóÖ ÎåÄÌñâ ÌïµÏã¨Í≥µÏïΩ Ï∂ïÏÇ∞ÎÜçÍ∞Ä Í≤ΩÏüÅ...                  Ï¢ÖÌï©
8  Ïù¥ÌôçÏû¨ Í∏∞Ïûê ÌïúÍ∞ïÏú†Ïó≠ÌôòÍ≤ΩÏ≤≠ÏùÄ ÌôòÍ≤ΩÎ∂ÄÍ∞Ä Ï†ÑÍµ≠ ÏßÄÏûêÏ≤¥Î•º ÎåÄÏÉÅÏúºÎ°ú Ïã§ÏãúÌïú Í≥µÍ≥µÌïòÏàòÎèÑÏùò Ïö¥...                  Ï¢ÖÌï©
9  ÏÇºÏÑ±Ï†ÑÏûêÍ∞Ä Í∞úÏ≤ôÌïú Í∏ÄÎ°úÎ≤å Ìè¥ÎçîÎ∏îÌè∞ ÏãúÏû•Ïóê ÌôîÏõ®Ïù¥Î™®ÌÜ†Î°úÎùºÍ∞Ä Í∞ÄÏÑ∏ÌïòÎ©¥ÏÑú ÌïúÏ§ë ÎßûÎåÄÍ≤∞Ïù¥ ...               IT,Í≥ºÌïô

In [ ]:
selected_columns7